# 1 - Source Text Filtering

In [133]:
import sys
print(sys.version)

3.5.2 |Continuum Analytics, Inc.| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


In [134]:
# Use AzureML's data collector to log various metrics!
#from azureml.logging import current_scriptrun
#logger = current_scriptrun()

import string, re
import pandas as pd
from nltk.corpus import stopwords
from gensim import corpora, models, similarities
import azureml
from __future__ import division
import numpy as np
import nltk
from nltk.corpus import stopwords
from azure.storage.blob import BlockBlobService

Imprt source text and write to a pandas dataframe

In [135]:
import os
os.getcwd()

'C:\\Users\\pattyry\\Documents\\AzureML\\NextAgenda_CodeStory\\NextAgenda_CodeStory'

In [136]:
import pickle
with open('Data2.pkl', 'rb') as f:
    data = pickle.load(f)
    print("Data unpickled")
    import pandas as pd
data = pd.DataFrame(data)
data.shape

Data unpickled


(32233, 16)

Review data

In [137]:
data.head(2)

,Ticker,Year,Month,Day,Date,Category,Category Number,Future_4_weeks,Future_26_weeks,Text,Label,Price_Date,Price_4_weeks,Price_26_weeks,Return_4_weeks,Return_26_weeks
1,AAL,2014,02,28,2014-02-28,Major Airlines,770,2014-03-28,2014-08-29,OverviewAmerican Airlines Group Inc. (AAG) a D...,Item1,35.798592,34.305775,37.784386,-4.178315,0.788191
2,AAL,2013,02,20,2013-02-20,Major Airlines,770,2013-03-20,2013-08-21,"Chapter 11 ProceedingsOn November 29, 2011 (th...",Item1,12.921616,16.702129,15.325636,25.062782,11.639949


Discard data not of interest

In [138]:
data1=data.drop(['Month', 'Day', 
                'Future_4_weeks', 'Future_26_weeks','Price_4_weeks','Price_26_weeks'], axis=1)
print(data1.shape)
data1.head(2)

(32233, 10)


,Ticker,Year,Date,Category,Category Number,Text,Label,Price_Date,Return_4_weeks,Return_26_weeks
1,AAL,2014,2014-02-28,Major Airlines,770,OverviewAmerican Airlines Group Inc. (AAG) a D...,Item1,35.798592,-4.178315,0.788191
2,AAL,2013,2013-02-20,Major Airlines,770,"Chapter 11 ProceedingsOn November 29, 2011 (th...",Item1,12.921616,25.062782,11.639949


In [139]:
data1['Year'].value_counts()

2014    4174
2015    4033
2012    3694
2013    3652
2016    3594
2011    2901
2010    2774
2009    2675
2017    2451
2008    2285
Name: Year, dtype: int64

In [140]:
data2=data1.drop(['Year','Date'], axis=1)
print('data shape',data2.shape)
data2.head(2)

data shape (32233, 8)


,Ticker,Category,Category Number,Text,Label,Price_Date,Return_4_weeks,Return_26_weeks
1,AAL,Major Airlines,770,OverviewAmerican Airlines Group Inc. (AAG) a D...,Item1,35.798592,-4.178315,0.788191
2,AAL,Major Airlines,770,"Chapter 11 ProceedingsOn November 29, 2011 (th...",Item1,12.921616,25.062782,11.639949


Create calculated normalized percent return, creaate 3 bins

In [141]:
data2["PercReturn_4_Weeks"] = data2.Return_4_weeks / data2.Price_Date
data2["PercReturn_26_Weeks"] = data2.Return_26_weeks / data2.Price_Date
data2["ReturnBin_4Weeks"] = pd.qcut(data2["PercReturn_4_Weeks"], 3,labels=range(3))
data2["ReturnBin_26Weeks"] = pd.qcut(data2["PercReturn_26_Weeks"], 3,labels=range(3))
referencedata=data2
print(data2.shape)
data2.head(2)


(32233, 12)


,Ticker,Category,Category Number,Text,Label,Price_Date,Return_4_weeks,Return_26_weeks,PercReturn_4_Weeks,PercReturn_26_Weeks,ReturnBin_4Weeks,ReturnBin_26Weeks
1,AAL,Major Airlines,770,OverviewAmerican Airlines Group Inc. (AAG) a D...,Item1,35.798592,-4.178315,0.788191,-0.116717,0.022017,0,1
2,AAL,Major Airlines,770,"Chapter 11 ProceedingsOn November 29, 2011 (th...",Item1,12.921616,25.062782,11.639949,1.939601,0.900812,2,2


In [142]:
data2.dtypes
data2 = data2.sort_values(['Return_4_weeks'], ascending=[False])
data2.head(2)

,Ticker,Category,Category Number,Text,Label,Price_Date,Return_4_weeks,Return_26_weeks,PercReturn_4_Weeks,PercReturn_26_Weeks,ReturnBin_4Weeks,ReturnBin_26Weeks
18777,BRN,Independent Oil & Gas,121,The business of Barnwell and its subsidiaries ...,Item1A,3.05,157.399339,101.735227,51.606341,33.355812,2,2
2237,BRN,Independent Oil & Gas,121,Overview Barnwell was incorporated in Delaware...,Item1,3.05,157.399339,101.735227,51.606341,33.355812,2,2


In [143]:
df=data2
df['fulltext'] = df[['Ticker', 'Text']].apply(lambda x: '.   '.join(x), axis=1)
df=df.drop(['Text','Ticker','Price_Date','Return_4_weeks','Return_26_weeks','PercReturn_4_Weeks','PercReturn_26_Weeks','Category Number','Label'], axis=1)
print(df.shape)
df.head(3)


(32233, 4)


,Category,ReturnBin_4Weeks,ReturnBin_26Weeks,fulltext
18777,Independent Oil & Gas,2,2,BRN. The business of Barnwell and its subsid...
2237,Independent Oil & Gas,2,2,BRN. Overview Barnwell was incorporated in D...
35290,Independent Oil & Gas,2,2,BRN. MANAGEMENTS DISCUSSION AND ANALYSIS OF...


In [144]:
df.dtypes

Category               object
ReturnBin_4Weeks     category
ReturnBin_26Weeks    category
fulltext               object
dtype: object

In [145]:
print(df.Category.unique())

['Independent Oil & Gas' 'Semiconductor - Integrated Circuits'
 'Oil & Gas Equipment & Services' 'Regional - Midwest Banks'
 'Oil & Gas Drilling & Exploration' 'Industrial Electrical Equipment'
 'Credit Services' '' 'REIT - Diversified' 'Shipping'
 'Beverages - Soft Drinks' 'Auto Dealerships' 'Mortgage Investment'
 'Regional - Pacific Banks' 'Asset Management' 'Business Services'
 'Money Center Banks' 'Metal Fabrication' 'General Building Materials'
 'Heavy Construction' 'Biotechnology' 'Application Software'
 'Networking & Communication Devices' 'Textile - Apparel Clothing'
 'Health Care Plans' 'Steel & Iron' 'Restaurants' 'Rubber & Plastics'
 'REIT - Hotel/Motel' 'Property & Casualty Insurance' 'Broadcasting - TV'
 'Technical & System Software' 'Diversified Electronics' 'Drug Stores'
 'Semiconductor Equipment & Materials' 'Internet Information Providers'
 'Tobacco Products' 'Regional - Mid-Atlantic Banks' 'Life Insurance'
 'Oil & Gas Pipelines' 'Specialty Chemicals' 'Trucks & Other V

In [146]:
with open('allindustriessource.pkl', 'wb') as f:
    pickle.dump(df, f)
#df = pd.read_pickle(file_name)

In [147]:

df1 = df[(df['Category'] =="Biotechnology")]
print(df1['Category'].value_counts())

Biotechnology    996
Name: Category, dtype: int64


In [148]:
df1=df1.drop(['Category'], axis=1)
df1.head(2)

,ReturnBin_4Weeks,ReturnBin_26Weeks,fulltext
44198,2,0,OGEN. MANAGEMENTS DISCUSSION AND ANALYSIS O...
11098,2,0,OGEN. 1


In [149]:
with open('biotechsource.pkl', 'wb') as f:
    pickle.dump(data3, f)
#df = pd.read_pickle(file_name)